In [1]:
import numpy
import scipy
import scipy.sparse
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
import sklearn.metrics.pairwise
import string
import collections

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from nltk import word_tokenize, regexp_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def process_text(text, stem=True):
    """ Tokenize text in sets of 2 words in addition to by word """
    text = text.translate(string.punctuation)
    words = word_tokenize(text)
    clean_words = []
    stopwds = set(stopwords.words('english'))
    for w in words:
        if w not in stopwds:
            clean_words.append(w)
    
    wordstr = " ".join(clean_words)
            
    setsOf2 = regexp_tokenize(wordstr, pattern = r"(?=\b([a-zA-Z]{1,40} [a-zA-Z]{1,40}))")

    tokens = setsOf2 + clean_words
    
    return tokens


In [4]:
descriptions = []

with open('descriptions.txt', encoding = "utf8") as f:
    for line in f:
        text = line.lower()                                       ## Lowercase all characters
        text = text.replace("[comma]"," ")                        ## Replace [commas] with empty space
        for ch in text:
            if ch < "0" or (ch < "a" and ch > "9") or ch > "z":   ## The cleaning operation happens here, remove all special characters
                text = text.replace(ch," ")
        text = ' '.join(text.split())                             ## Remove double spacing from sentences
        descriptions.append(text)

dataSet = numpy.array(descriptions)

In [1]:
vectorizer = TfidfVectorizer(tokenizer=process_text, stop_words='english')
TfIdf_dataSet = vectorizer.fit_transform(dataSet)
#print("What our Tf-Idf looks like: ","\n")
#print(TfIdf_dataSet[0:1])

#print(vectorizer.get_feature_names(),"\n")

vectorVocab = vectorizer._validate_vocabulary()

NameError: name 'TfidfVectorizer' is not defined

In [7]:
cosineSimilarity = sklearn.metrics.pairwise.cosine_similarity(TfIdf_dataSet)
print(cosineSimilarity)

[[1.         0.02193248 0.00499729 ... 0.00219639 0.01612756 0.03193304]
 [0.02193248 1.         0.02312615 ... 0.01646629 0.02315855 0.01587485]
 [0.00499729 0.02312615 1.         ... 0.00198232 0.02222558 0.00950195]
 ...
 [0.00219639 0.01646629 0.00198232 ... 1.         0.00165995 0.00170524]
 [0.01612756 0.02315855 0.02222558 ... 0.00165995 1.         0.02644732]
 [0.03193304 0.01587485 0.00950195 ... 0.00170524 0.02644732 1.        ]]


In [10]:
numpy.fill_diagonal(cosineSimilarity,1.1)
cosineSimilaritySorted = numpy.argsort((-1*(cosineSimilarity)),axis=1)
#print(cosineSimilaritySorted)
#cosineSimilaritySorted = numpy.argsort((-1*(cosineSimilarity)),axis=1)
top5similar = (cosineSimilaritySorted[:,0:5])
print()
print(top5similar)


[[   0   65 1454 1047 1085]
 [   1  556  549   97  593]
 [   2  342   28  379 1417]
 ...
 [1477 1372  530  210   37]
 [1478  967  706 1276 1038]
 [1479 1144  773  504  861]]


In [11]:
numpy.savetxt("results.csv", top5similar.astype(int), fmt='%i', delimiter=",")